# Data mining the Gaia star archive
### Question: The Gaia database contains billions of stars. Using data mining techniques, identify groups of stars that belong together
Stars that formed from the same cloud, older clusters broke apart - more difficult to classify. Use unsupervised machine learning algorithms to classify star clusters.

Gaia archive: https://gea.esac.esa.int/archive/

Data explained: https://www.cosmos.esa.int/web/gaia/dr2

Gaia data set two was taken on 25 July 2014 (10:30 UTC) and 23 May 2016

In [1]:
import pandas as pd
df = pd.read_csv("GaiaSource_000-020-110.csv")
stats = df.describe()
stats

,solution_id,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,...,scan_direction_mean_k3,scan_direction_mean_k4,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,l,b,ecl_lon,ecl_lat
count,1.705790e+05,1.705790e+05,1.705790e+05,170579.0,170579.000000,170579.000000,170579.000000,170579.000000,403.000000,403.000000,...,170579.000000,170579.000000,170579.000000,1.705790e+05,170579.000000,170579.000000,170579.000000,170579.000000,170579.000000,170579.000000
mean,1.635378e+18,6.916184e+18,5.710462e+08,2015.0,314.445730,9.989248,-2.558527,5.915029,2.842900,0.633141,...,-4.029097,27.643187,57.872288,1.374082e+04,26.393011,18.201054,46.092700,-29.074272,316.145533,14.042220
std,0.000000e+00,7.824116e+14,3.303331e+08,0.0,1.494900,10.504467,0.971403,5.701347,3.065102,0.212730,...,28.123823,1.589400,15.012840,3.072943e+05,1058.442394,1.811302,1.441368,1.257351,1.575566,0.914638
min,1.635378e+18,6.914851e+18,1.177300e+04,2015.0,310.788140,0.148633,-4.775293,0.103670,-1.791382,0.224590,...,-59.998318,20.475262,11.000000,1.258887e+01,0.546064,6.296060,42.849223,-31.893663,312.234516,11.766862
25%,1.635378e+18,6.915492e+18,2.840771e+08,2015.0,313.376715,3.436794,-3.331537,1.383367,1.155185,0.460758,...,-24.187854,26.690452,48.000000,2.321637e+02,2.780138,17.234616,44.938345,-30.102178,315.080043,13.390512
50%,1.635378e+18,6.916201e+18,5.700315e+08,2015.0,314.614848,4.602855,-2.642237,4.267861,1.996829,0.638235,...,-3.065681,27.744164,59.000000,5.945685e+02,3.730346,18.589265,46.105692,-29.023634,316.303455,14.025367
75%,1.635378e+18,6.916868e+18,8.582401e+08,2015.0,315.537785,13.754998,-1.843954,9.027622,3.569003,0.817786,...,11.675235,28.743473,69.000000,2.070435e+03,6.488504,19.610284,47.280949,-28.089637,317.342888,14.697067
max,1.635378e+18,6.917529e+18,1.142677e+09,2015.0,317.798057,91.395674,-0.002570,84.366297,23.499923,0.994737,...,59.995136,35.308086,114.000000,4.914554e+07,308189.671077,22.774803,48.915665,-26.394036,319.488987,16.333882


In [2]:
#glimps of the data
df.head(5) #df[0:5] does the same thing

,solution_id,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,...,scan_direction_mean_k4,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_variable_flag,l,b,ecl_lon,ecl_lat
0,1635378410781933568,6914851441950097280,644618244,2015.0,312.115755,3.956164,-3.740349,1.404508,NaN,NaN,...,28.036676,68,4444.580030,4.916518,16.405193,NOT_AVAILABLE,43.620180,-27.630762,313.504567,13.567278
1,1635378410781933568,6914851476309836160,668015830,2015.0,312.134332,4.045969,-3.739628,1.437343,NaN,NaN,...,28.035465,69,5137.131412,7.556112,16.247968,NOT_AVAILABLE,43.630971,-27.646661,313.523107,13.562885
2,1635378410781933568,6914851476309836800,609250353,2015.0,312.138582,25.938898,-3.737056,11.683244,NaN,NaN,...,23.329552,33,416.763565,3.129720,18.975046,NOT_AVAILABLE,43.635821,-27.649139,313.528028,13.564193
3,1635378410781933568,6914851476309838848,393393107,2015.0,312.131173,24.819271,-3.735257,11.175843,NaN,NaN,...,23.574673,31,353.162079,3.602725,19.154835,NOT_AVAILABLE,43.633581,-27.641793,313.521223,13.567953
4,1635378410781933568,6914851476309839872,241639886,2015.0,312.134676,23.953915,-3.733854,10.811309,NaN,NaN,...,23.329810,33,251.991138,3.906820,19.521307,NOT_AVAILABLE,43.636869,-27.644181,313.525077,13.568342


In [3]:
N = len(df)
print('Data loaded. Contains {} samles.'.format(N))

Data loaded. Contains 170579 samles.


In [4]:
cols = list(df)
print('Data contains {} columns'.format(len(cols)))

Data contains 57 columns


In [5]:
N = df.shape[0]
P = df.shape[1]

print('Data has {} rows, {} columns'.format(N, P))

Data has 170579 rows, 57 columns


### How many parallaxes are negative?

In [6]:
import math

#greater than 0 parallax
above = df[ 0 < df.parallax]
#less than 0 parallax
below = df[ 0 >= df.parallax]

a = above.shape[0]
b = below.shape[0]

print(a)
print(b)

b / (b+a)

385
18


0.04466501240694789

In [7]:
#Shows each column and the type of each
df.dtypes #no spectra data

solution_id                        int64
source_id                          int64
random_index                       int64
ref_epoch                        float64
ra                               float64
ra_error                         float64
dec                              float64
dec_error                        float64
parallax                         float64
parallax_error                   float64
pmra                             float64
pmra_error                       float64
pmdec                            float64
pmdec_error                      float64
ra_dec_corr                      float64
ra_parallax_corr                 float64
ra_pmra_corr                     float64
ra_pmdec_corr                    float64
dec_parallax_corr                float64
dec_pmra_corr                    float64
dec_pmdec_corr                   float64
parallax_pmra_corr               float64
parallax_pmdec_corr              float64
pmra_pmdec_corr                  float64
astrometric_n_ob